In [ ]:
import numpy as np
import scipy.optimize as spo
import matplotlib.pyplot as plt

In [ ]:
#set of functions
#Building non-linear grid
def non_lin_grid(lenght, xN, coef):
    
    #lenght - [m]
    # xN - number of cells
    xL=np.float64(0.)
    xR=xL+(lenght/L0)
    H=xR
    
    #uniformly distributed parameter
    ksi = np.zeros(xN+1)
    
    for i in range(np.int(xN)):
        
        ksi[i+1] = ksi[i] + one/xN
            
    tmpr=((coef+one)/(coef-one))**(one-ksi)
    #current x has 1001 points!!!! - the edges
    x=H*(coef+one-(coef-one)*tmpr)/(one+tmpr)
    
    #size of each cell
    dx=(x[1:xN+1]-x[0:xN])
    
    #now x - array of coordinates of cells
    x = np.zeros(xN)
    x[0] = xL+half*dx[0]
    
    for i in range(dx.size-1):
        
        x[i+1] = x[i] + half*(dx[i+1]+dx[i])
        
    return x, dx

def set_w(dx):

    xN = dx.size
    
    #%линейная двухточечная аппроксимация нелинейной сетки
    w_r = dx[0:xN-1]/(dx[0:xN-1]+dx[1:xN])                        #    !линейная аппроксимация внутри области П= mass_r(1)*П2+mass_l(1)*П1; П1,П2-значения в ячейках П-между ячейками
    w_l = dx[1:xN]/(dx[0:xN-1]+dx[1:xN])

    we_l = np.zeros(2)
    we_r = np.zeros(2)
    we_r[0]=-(dx[xN-1]/(dx[xN-2]+dx[xN-1]))                             #  !линейная аппроксимация на правой границе области П= edge_r(1)*П1+edge_r(2)*П2; П1,П2-значения в ячейках П-на правая грань
    we_r[1]=((dx[xN-2]+two*dx[xN-1])/(dx[xN-2]+dx[xN-1])) 

    we_l[0] = (two*dx[0]+dx[1])/(dx[0]+dx[1])                       # !линейная аппроксимация на левой границе области П= edge_l(1)*П1+edge_l(2)*П2; П1,П2-значения в ячейках П-на левой грани
    we_l[1] = -dx[0]/(dx[0]+dx[1])

    
    return w_r, w_l, we_r, we_l

def gen_dx_w(x):
    
    dx = np.zeros(x.shape)
    
    dx[0] = two*x[0]
    
    for i in range(1, x.size):
        
        dx[i] = two*(x[i] - x[i-1] - half*dx[i-1])
    
#     print dx[:10]
    
    w_r, w_l, we_r, we_l = set_w(dx)

    return dx, w_r, w_l, we_r, we_l

In [ ]:
#Compute Jacobian
def get_Jac(AD, AJ, IRc, n, Th, T1, dx, w_r, w_l, we_r, we_l):
    
    Jac_f = np.zeros([4,4,n.shape[1]])
#     AD = np.zeros([4,4,n.shape[1]])
#     AJ = np.zeros([4,n.shape[1]])
#     print Jac_f.shape
#     print IRc.shape
#     print AD.shape
#     print n.shape
    Jac_f [0,0,:] = -IRc[0,1,:]*n[0,:]
    Jac_f [1,0,:] = -IRc[0,0,:]*n[0,:]
    Jac_f [2,0,:] = IRc[0,1,:]*n[0,:]
    
    Jac_f [0,1,:] = two*IRc[1,1,:]*n[4,:]*n[0,:] - IRc[0,1,:]*n[2,:]
    Jac_f [1,1,:] = -IRc[0,0,:]*(n[2,:]+n[3,:]+n[4,:]+n[5,:] - (P/Th) + (T1*n[0,:])/Th)-IRc[1,0,:]*n[0,:]*n[0,:] - two*IRc[1,0,:]*n[0,:]*n[3,:] - IRc[0,0,:]*n[0,:]*((T1/Th)+one)
    Jac_f [2,1,:] = IRc[0,1,:]*n[2,:] - IRc[0,2,:]*n[4,:] - two*IRc[1,1,:]*n[0,:]*n[4,:] + two*IRc[1,2,:]*n[0,:]*n[5,:]
    Jac_f [3,1,:] = IRc[0,2,:]*n[4,:] - two*IRc[1,2,:]*n[0,:]*n[5,:] 
    
    Jac_f [0,2,:] = IRc[1,1,:]*n[0,:]*n[0,:] - IRc[0,1,:]*n[2,:] +two*IRc[1,1,:]*n[0,:]*n[4,:]
    Jac_f [1,2,:] = -IRc[0,0,:]*(n[2,:]+n[3,:]+n[4,:]+n[5,:] - (P/Th) + (T1*n[0,:])/Th) - two*IRc[1,0,:]*n[0,:]*n[3,:] - IRc[0,0,:]*n[0,:]*((T1/Th)+one)
    Jac_f [2,2,:] = IRc[0,1,:]*n[2,:] - IRc[0,2,:]*n[0,:] - IRc[1,1,:]*n[0,:]*n[0,:] - IRc[0,2,:]*n[4,:] - two*IRc[1,1,:]*n[0,:]*n[4,:] + two*IRc[1,2,:]*n[0,:]*n[5,:] 
    Jac_f [3,2,:] = IRc[0,2,:]*n[4,:] + IRc[0,2,:]*n[0,:] - two*IRc[1,2,:]*n[0,:]*n[5,:] 
    
    Jac_f [0,3,:] = - two*IRc[0,1,:]*n[2,:] +four*IRc[1,1,:]*n[0,:]*n[4,:]
    Jac_f [1,3,:] = -two*IRc[0,0,:]*(n[2,:]+n[3,:]+n[4,:]+n[5,:] - (P/Th) + (T1*n[0,:])/Th) - four*IRc[1,0,:]*n[0,:]*n[3,:] - IRc[0,0,:]*n[0,:]*((two*T1/Th)+one)
    Jac_f [2,3,:] = IRc[1,2,:]*n[0,:]*n[0,:] + two*IRc[0,1,:]*n[2,:] - two*IRc[0,2,:]*n[4,:] - four*IRc[1,1,:]*n[0,:]*n[4,:] + four*IRc[1,2,:]*n[0,:]*n[5,:] 
    Jac_f [3,3,:] = two*IRc[0,2,:]*n[4,:] - IRc[1,2,:]*n[0,:]*n[0,:] - four*IRc[1,2,:]*n[0,:]*n[5,:] 
    
    return Jac_f